# Multiclass Prediction in Deep Learning Models

## Read in the Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [3]:
df = pd.read_csv('../Resources/meet_or_beat.csv')

In [4]:
df.shape

(71968, 5)

In [5]:
df.head(3)

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns
0,2.01,1.67,11.0,0.051444,0.018585
1,0.17,0.19,6.0,0.112955,-0.000510
2,-0.07,0.14,4.0,0.077167,-0.046104


## Preparing the Data

In [6]:
# Generate the categorical outcome variable
df['earnings_outcome'] = np.nan
df.loc[(df['EPS']==df['forecasted_eps']), 'earnings_outcome'] = 'meet'
df.loc[(df['EPS']>df['forecasted_eps']), 'earnings_outcome'] = 'beat'
df.loc[(df['EPS']<df['forecasted_eps']), 'earnings_outcome'] = 'lose'
df.head()

,EPS,forecasted_eps,noOfEsts,after_total_returns,before_total_returns,earnings_outcome
0,2.01,1.67,11.0,0.051444,0.018585,beat
1,0.17,0.19,6.0,0.112955,-0.000510,lose
2,-0.07,0.14,4.0,0.077167,-0.046104,lose
3,0.48,0.51,8.0,-0.006130,-0.004899,lose
4,-0.24,-0.27,9.0,0.089762,-0.025466,beat


In [7]:
# Preview the output variable 
y = df['earnings_outcome']
y

0        beat
1        lose
2        lose
3        lose
4        beat
         ... 
71963    beat
71964    beat
71965    lose
71966    beat
71967    meet
Name: earnings_outcome, Length: 71968, dtype: object

In [8]:
# Encode earnings labels to integers
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
encoded_y

array([0, 1, 1, ..., 1, 0, 2])

In [9]:
# Save the unique number of labels for future use
number_of_classes = len(list(y.drop_duplicates()))
number_of_classes

3

In [10]:
# Convert labeled integers to a Keras `categorical` data type
from keras.utils.np_utils import to_categorical
y_categorical = to_categorical(encoded_y, num_classes=number_of_classes)


In [11]:
# Specify X (predictor) variables
X = df[['forecasted_eps',
              'before_total_returns','noOfEsts']]
X.head(3)

,forecasted_eps,before_total_returns,noOfEsts
0,1.67,0.018585,11.0
1,0.19,-0.000510,6.0
2,0.14,-0.046104,4.0


In [12]:
# Split into training and testing windows
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, random_state=1)

In [13]:
# Preview the encoded data we're trying to predict
pd.DataFrame(y_train).head(5)

,0,1,2
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,1.0,0.0,0.0


In [14]:
# Check for class balance
pd.DataFrame(y_train).sum()

0    32924.0
1    16878.0
2     4174.0
dtype: float32

In [15]:
# Save the count of unique predictor variables for use in model
number_of_predictors = len(X.columns)
len(X.columns)

3

## Building, Fitting and Predicting with Multiple Classes

In [16]:
# Build the neural network layers
model = Sequential()
model.add(Dense(9, input_dim=number_of_predictors, activation='relu'))
model.add(Dense(6, activation='relu'))

Metal device set to: Apple M1 Max


2022-03-10 12:40:19.925247: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-10 12:40:19.925372: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [17]:
# Add the final output layer
# With the # of possible outputs equal to the number classes 
model.add(Dense(number_of_classes, activation='softmax'))

In [18]:
# Compile the model (with multi-class specific parameters)
model.compile(loss="categorical_crossentropy", 
              optimizer= "adam", 
              metrics=['categorical_accuracy'])


In [19]:
# Summarise the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 36        
                                                                 
 dense_1 (Dense)             (None, 6)                 60        
                                                                 
 dense_2 (Dense)             (None, 3)                 21        
                                                                 
Total params: 117
Trainable params: 117
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Fit the model
model.fit(X_train,y_train, 
                    epochs=20,
                    batch_size=1000,
                    shuffle=True)

Epoch 1/20


2022-03-10 12:40:20.078758: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-10 12:40:20.221421: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


54/54 [==============================] - 1s 7ms/step - loss: 1.1016 - categorical_accuracy: 0.3447
Epoch 2/20
54/54 [==============================] - 0s 7ms/step - loss: 0.9523 - categorical_accuracy: 0.5951
Epoch 3/20
54/54 [==============================] - 0s 7ms/step - loss: 0.8961 - categorical_accuracy: 0.6068
Epoch 4/20
54/54 [==============================] - 0s 7ms/step - loss: 0.8767 - categorical_accuracy: 0.6064
Epoch 5/20
54/54 [==============================] - 0s 8ms/step - loss: 0.8689 - categorical_accuracy: 0.6057
Epoch 6/20
54/54 [==============================] - 0s 7ms/step - loss: 0.8633 - categorical_accuracy: 0.6058
Epoch 7/20
54/54 [==============================] - 0s 7ms/step - loss: 0.8582 - categorical_accuracy: 0.6075
Epoch 8/20
54/54 [==============================] - 0s 7ms/step - loss: 0.8541 - categorical_accuracy: 0.6094
Epoch 9/20
54/54 [==============================] - 0s 7ms/step - loss: 0.8517 - categorical_accuracy: 0.6093
Epoch 10/20
54/54 [==

In [21]:
# Evaluate model on the test data
model.evaluate(X_test,y_test, verbose=2)

2022-03-10 12:40:28.137995: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


563/563 - 3s - loss: 0.8393 - categorical_accuracy: 0.6155 - 3s/epoch - 5ms/step


[0.8393334746360779, 0.6154958009719849]

In [22]:
# Save predictions on the test data
predictions = model.predict(X_test)
predictions

2022-03-10 12:40:31.012702: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


array([[0.5837806 , 0.33498585, 0.08123355],
       [0.8119511 , 0.14069377, 0.04735518],
       [0.54987043, 0.36816022, 0.08196934],
       ...,
       [0.6281463 , 0.29597   , 0.07588373],
       [0.558863  , 0.35280615, 0.08833086],
       [0.5180365 , 0.40867785, 0.07328575]], dtype=float32)

In [23]:
# Get the most likely prediction for each observation
most_likely = np.argmax(predictions, axis=1)
most_likely

array([0, 0, 0, ..., 0, 0, 0])

In [24]:
# Convert most likely category back to original labels
most_likely = encoder.inverse_transform((most_likely))
most_likely

array(['beat', 'beat', 'beat', ..., 'beat', 'beat', 'beat'], dtype=object)

In [25]:
# Evaluate prediction balance
pd.DataFrame(most_likely).value_counts()

beat    17934
lose       58
dtype: int64